In [9]:
pip install --upgrade transformers torch


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 37.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 48.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1
    Uninstalling torch-2.3.1:
      Successfully uninstalled torch-2.3.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch

# Check if a CUDA-enabled GPU is available or if using MPS (for macOS with M1/M2 chip)
use_fp16 = torch.cuda.is_available()
use_fp16

False

In [1]:
import torch

# Determine if CUDA or MPS is available
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch.utils.data import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
import torch.nn.functional as F
from transformers import TrainingArguments


# Download NLTK stop words
nltk.download('stopwords')
from nltk.corpus import stopwords

# Load the dataset
file_path = "/Users/aaryanshah/Oncampus-Job/NLP_Gal/data/TrainingSet 1(Deals).csv"
df = pd.read_csv(file_path)

# Define text columns and the target column
text_columns = ['Target Business Description', 'M&A Headline', 'Deal Synopsis']
df['combined_text'] = df[text_columns].apply(lambda x: ' '.join(x.dropna()), axis=1)

# Clean the text data
def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['combined_text'].apply(clean_text)

# Define score columns
score_columns = ['Singh', 'Arora', 'Neilsen', 'RH', 'Uche', 'Edrick']

# Function to remove the outlier from each row using manual iteration to find the max difference
def remove_outlier_manual(row):
    scores = row[score_columns].dropna()  # Remove NaNs for processing
    if len(scores) > 1:  # Ensure there are enough scores to compare
        median = np.median(scores)
        diffs = np.abs(scores - median)
        max_diff_value = diffs.max()
        max_diff_index = diffs[diffs == max_diff_value].index[0]
        row.at[max_diff_index] = np.nan
    return row

# Apply the outlier removal function
df_cleaned = df.apply(remove_outlier_manual, axis=1)

# Calculate the average for each row
df_cleaned['Average'] = df_cleaned[score_columns].mean(axis=1)

# Handle NaN values in the Average column
df_cleaned = df_cleaned.dropna(subset=['Average'])

# Ensure 'Average' values are within 1 to 5
df_cleaned = df_cleaned[df_cleaned['Average'].between(1, 5)]

# Map the 'average' scores to ensure they are within 1 to 5 range
target_column = 'Average'
df_cleaned['labels'] = df_cleaned[target_column].astype(int) - 1  # Make sure labels are 0-indexed for classification


# Additional Domain-Specific Keyword Lists
core_keywords = set([
    "crude", "shale", "hydrocarbon", "methane", "petroleum", "gas", "oil",
    "gasoline", "diesel", "ethanol", "propane", "petrol", "coke", "gasohol",
    "coal", "lng", "barrel", "condensate", "benzene", "bitumen", "asphalt",
    "coking", "butane", "kerosene", "bopd", "butene", "fossil", "liqufied",
    "dlng", "fuel", "engine", "oilgas", "petrolium", "petroli", "petroleos",
    "gasrelated", "gasrec", "petroluem", "petrolera", "gallon", "gasfueled",
    "gasinvest", "water", "fluid", "liquide", "grease", "liquor", "rock",
    "gasket", "condes", "octanex", "toilet", "carbide", "catalytic", "vapor",
    "coolant", "boilers", "hydrogenfuelled", "lignite", "pine", "pollution",
    "pygas", "mortar", "exxon", "exxonmobil", "chevron", "cement", "waste",
    "concrete", "hydrogen", "liquefied", "hydrothermal", "liquid", "sewage",
    "plastic", "polypropylene", "polyethylene", "polymer", "propylene",
    "polystyrene", "polyester", "copolymer", "polyvinyl", "xylene", "polysterene",
    "polyurethane", "polyethlyene", "polystyrol", "polythene", "polyprophylene",
    "polythylene", "polybutylene", "polyblend", "petrolube", "corplubricants",
    "petroquimica", "chemical", "lubricant", "petrochemical", "compound",
    "pharmaceutical", "fertilizer", "ethylene", "pvc", "additive", "catalyst",
    "lubricating", "olefin", "lube", "butadiene", "aromatics", "thermoplastic",
    "textile", "solvent", "thiochemicals", "bauxite", "mtbe", "acetylene",
    "oleochemicals", "acetate", "ammonia", "ammonium", "aromatic", "bicarbonate",
    "biobutanol", "chlorochemicals", "ethylbenzene", "fertiliser", "herbicide",
    "isophtheric", "isopropyl", "oleochemical", "mbpd", "petrochemistry",
    "phosphate", "phosphorus", "butadine", "cfc", "lubricants", "olefins",
    "polymerization", "pyrolysis", "chemicals", "inorganic", "organic",
    "synthetic", "composite", "lubricated", "hydrated", "hydrochloric",
    "nitrogenous", "reaction", "drug", "hydrate", "foam", "synthesis",
    "chemistry", "hydrogenrich", "minerales", "contaminant", "mineral",
    "uranium", "silicate", "production", "manufacture", "produce",
    "construction", "distributor", "marketing", "supply", "building",
    "producing", "manufacturing", "trading", "advertising", "supplying",
    "constructing", "selling", "fabrication", "purchasing", "procurement",
    "sourcing", "develop", "developing", "sell", "maintenance", "consulting",
    "handling", "distributing", "welding", "imaging", "repairing", "billing",
    "sales", "hr", "wholesale", "business", "manufacturer", "industrial",
    "technology", "industry", "market", "producer", "financial", "acquisition",
    "produced", "vendor", "acquires", "revenue", "marketed", "industriale",
    "factory", "build", "stock", "purchase", "processed", "enterprise",
    "trader", "constructed", "supplier", "industria", "making", "make",
    "built", "engineered", "composed", "acquiring", "industrials",
    "corpfilling", "produkte", "corporate", "delivering", "development",
    "equipment", "establishment", "management", "operate", "operates",
    "operating", "operation", "service", "software", "services", "design",
    "maintaining", "manage", "managed", "manages", "developer", "machinery",
    "repair", "commodity", "servicing", "distributes", "specializing",
    "coordination", "designing", "distribute", "exchange", "growth",
    "ltdmarketing", "machining", "maintain", "managing", "operational",
    "partnership", "farming", "program", "proprietary", "woodworking",
    "deves", "strategic", "training", "assembly", "resale", "resells",
    "tech", "branded", "commercially", "contracting", "cooperative",
    "developement", "organizing", "engineer", "finishing", "formation",
    "foundry", "programs", "quality", "operations", "planning", "productive",
    "prospecting", "merger", "inventory", "company", "gmbhs", "llc", "corp",
    "subsidiary", "corporation", "warehousing", "transportation", "vehicle",
    "transport", "automotive", "delivery", "logistics", "cargo", "car",
    "shipping", "truck", "packing", "hauling", "railway", "logistical",
    "trucking", "harbor", "seaport", "distribution", "storage", "port",
    "container", "warehouse", "bunker", "ship", "maersk", "railroad",
    "tankers", "sea", "coast", "marine", "submarine", "maritime", "travel",
    "automobile", "transit", "passenger", "ford", "freight", "parcel",
    "bus", "drive", "truckstop", "tow", "destination", "vehicular", "traffic",
    "towing", "transporte", "facility", "infrastructure", "transmission",
    "packaging", "aircraft", "courier", "subway", "trucks", "carrier",
    "deliver", "highway", "postal", "route", "drives", "airport", "place",
    "stations", "airborne", "package", "tanker", "transportadora",
    "trolley", "vessel", "carport", "dealership", "fleet", "shipment",
    "carriers", "facilities", "cars", "landfill", "military", "crew",
    "tacoma", "airline", "offshore", "aviation", "retail", "retailer",
    "supermarket", "shop", "shopping", "store", "station", "retailing",
    "sale", "merchant", "commerce", "customer", "wholesaler", "dealer",
    "catalogue", "cataloguer", "shopsouvenir", "refinery", "gasification",
    "distillation", "cracking", "refueling", "fueling", "exploration",
    "mining", "refining", "engineering", "processing", "drilling",
    "midstream", "downstream", "refine", "upstream", "extraction",
    "regasification", "refineria", "discovery", "exploring", "pumping",
    "explore", "explores", "drill", "drilled", "refinement", "surveying",
    "sensing", "lcm", "directional", "rotary", "blowout", "alkylation",
    "isomerization", "stratigraphy", "petrology", "sedimentology",
    "stratigraphic", "hydrocracking", "hydrotreating", "hydrodesulfurization",
    "geology", "sagd", "dehydrogenation", "css", "fcc", "psa", "drainage",
    "refined", "sands", "mapping", "process", "study", "hydraulic", "flow",
    "geological", "refines", "blending", "remediation", "reprocess",
    "concentrated", "convey", "pressure", "expertise", "processes",
    "project", "studies", "gi", "limestone", "liquefying", "plating",
    "scientific", "ground", "seismic", "sand", "pipeline", "oilfield",
    "pipe", "rig", "reservoir", "wellhead", "well", "pipelines", "borehole",
    "lidar", "wellbore", "drillships", "rigs", "bops", "eor", "mwd", "lwd",
    "ipr", "basin", "farm", "lp", "tube", "pumped", "tubing", "inflow",
    "tank", "farms", "creek", "deepwater", "rifle", "valve", "leak",
    "cylinder", "cavern", "coalbed", "sink", "ductor", "finpipe",
    "shalebased", "tanks", "pumps", "mud", "piped", "piston", "plumbing",
    "pump", "combustion", "gasfired", "energy", "electric", "electricity",
    "mw", "battery", "turbine", "capacity", "heating", "megawatt",
    "cogeneration", "electrical", "grid", "generator", "gigawatthours",
    "highvoltage", "inverters", "turbines", "generated", "generates",
    "furnace", "unit", "heat", "thermal", "compressor", "efficiency",
    "generating", "galvanized", "regulator", "outlet", "exhaust", "units",
    "furnaces", "power", "inverter", "powergen", "powertrain", "propulsion",
    "micropower", "molten", "tesla", "emission", "reactors"

    # Add more core keywords as necessary...
])

emerging_keywords = set([
    "solar", "photovoltaic", "solarbased", "solarworld", "solarpack",
    "solares", "solarkonzept", "sunlight", "sunpower", "solarbridge",
    "wind", "renewable", "renewables", "sustainable", "hydroelectric",
    "cleantech", "wave", "hydro", "recycled", "lightsource", "ecoplastic",
    "windpower", "eco", "ecooils", "seaenergy", "tidal", "electrolysis",
    "otec", "ocean", "plant", "pv", "hydros", "energetici", "natureworks",
    "terra", "atmospheric", "cloud", "energetica", "energeticas",
    "environment", "environmentally", "plants", "nature", "environmental",
    "nuclear", "biofuel", "biofuels", "bioenergia", "biocatalyst",
    "biological", "bioenergy", "bio", "bioethanol", "biosev",
    "regenerative", "bioengineering", "biopower", "regeneration",
    "biogaz", "biotech", "wastetoenergy", "bioleo", "bioforming",
    "bioproducts", "bioreactor", "biochem", "biosolutions", "biogas",
    "biomass", "biodiesel", "sugarcane", "biorefineries", "biochemical",
    "biochemtex", "biogazowa", "bioprocesses", "biorefinery", "lfg",
    "biotechnology", "botanical", "enzyme"
    # Add more emerging keywords as necessary...
])

# Create Features Based on Keyword Presence
def keyword_features(text):
    text_set = set(text.split())  # Convert text to a set of words
    core_count = len(core_keywords & text_set)
    emerging_count = len(emerging_keywords & text_set)
    return pd.Series([core_count, emerging_count])

df_cleaned[['core_count', 'emerging_count']] = df_cleaned['cleaned_text'].apply(keyword_features)

# Pre-tokenize the entire dataset (including additional features)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
max_len = 512
tokenized_texts = df_cleaned['cleaned_text'].apply(
    lambda x: tokenizer.encode_plus(
        x,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
).tolist()

# Convert tokenized data into a custom dataset
class CustomTextDataset(Dataset):
    def __init__(self, tokenized_data, labels, core_counts, emerging_counts):
        self.input_ids = torch.stack([item['input_ids'].squeeze(0) for item in tokenized_data])
        self.attention_mask = torch.stack([item['attention_mask'].squeeze(0) for item in tokenized_data])
        self.labels = torch.tensor(labels)
        self.core_counts = torch.tensor(core_counts, dtype=torch.float32)
        self.emerging_counts = torch.tensor(emerging_counts, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx],
            'core_count': self.core_counts[idx],
            'emerging_count': self.emerging_counts[idx]
        }

# Model with modified forward method to use core and emerging keyword counts

class CustomRobertaForSequenceClassification(RobertaForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, labels=None, core_count=None, emerging_count=None):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # Ensure that labels, core_count, and emerging_count are correctly broadcasted
        if labels is not None:
            labels_expanded = labels.unsqueeze(-1).expand_as(logits)  # Make sure labels are broadcasted correctly

            # Enhance logits adjustments with differentiated weighting based on class performance insights
            logits_adjustment_core = 0.1 + 0.05 * (labels_expanded == 4).float() + 0.05 * (labels_expanded == 5).float()
            logits_adjustment_emerging = 0.1 + 0.05 * (labels_expanded == 4).float() + 0.05 * (labels_expanded == 5).float()

            logits += core_count.unsqueeze(1) * logits_adjustment_core
            logits += emerging_count.unsqueeze(1) * logits_adjustment_emerging

        loss = None
        if labels is not None:
            custom_weights = torch.tensor([1.0, 1.2, 1.2, 1.5, 2.0], dtype=torch.float).to(logits.device)
            loss_fct = torch.nn.CrossEntropyLoss(weight=custom_weights)
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else logits



# Cross-validation setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold = 0

# Prepare results storage
all_fold_results = []

# Training arguments
# Modify the TrainingArguments without FP16
training_args = TrainingArguments(
    output_dir='./results',  # Directory for storing logs and model checkpoints
    num_train_epochs=7,  # Train for 7 epochs
    per_device_train_batch_size=8,  # Smaller batch size to manage GPU memory
    per_device_eval_batch_size=16,  # Batch size for evaluation
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,  # Alternatively, set warmup as a ratio of total steps
    weight_decay=0.01,  # Weight decay for regularization
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=50,  # Log model predictions every 50 steps
    evaluation_strategy="epoch",  # Evaluate model at the end of each epoch
    learning_rate=2e-5,  # Higher initial learning rate
    lr_scheduler_type='linear',  # Learning rate scheduler
    fp16=False,  # Disable mixed precision training
    gradient_accumulation_steps=2,  # Use gradient accumulation for larger effective batch size
    save_strategy="epoch",  # Save the model at the end of each epoch
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="eval_loss",  # Use evaluation loss to determine the best model
    save_total_limit=2  # Limit the total amount of checkpoints; older ones are deleted
)



for train_index, test_index in skf.split(df_cleaned['cleaned_text'], df_cleaned['labels']):
    fold += 1
    print(f"\nFold {fold}")

    # Create datasets
    train_dataset = CustomTextDataset(
        [tokenized_texts[i] for i in train_index],
        df_cleaned['labels'].iloc[train_index].tolist(),
        df_cleaned['core_count'].iloc[train_index].tolist(),
        df_cleaned['emerging_count'].iloc[train_index].tolist()
    )
    test_dataset = CustomTextDataset(
        [tokenized_texts[i] for i in test_index],
        df_cleaned['labels'].iloc[test_index].tolist(),
        df_cleaned['core_count'].iloc[test_index].tolist(),
        df_cleaned['emerging_count'].iloc[test_index].tolist()
    )

    # Initialize a fresh model for each fold with weighted loss
    model = CustomRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

    # Trainer with early stopping callback
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=lambda p: {
            'accuracy': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1)),
            'f1': classification_report(p.label_ids, np.argmax(p.predictions, axis=1), target_names=[str(i+1) for i in range(5)], output_dict=True)['weighted avg']['f1-score']
        },
        callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()
    print(f"Fold {fold} - Accuracy: {eval_result['eval_accuracy']}")
    print(f"Fold {fold} - F1-Score: {eval_result['eval_f1']}")

    all_fold_results.append({
        'fold': fold,
        'accuracy': eval_result['eval_accuracy'],
        'f1': eval_result['eval_f1']
    })

# Display results for all folds
for result in all_fold_results:
    print(f"Fold {result['fold']} - Accuracy: {result['accuracy']}, F1-Score: {result['f1']}")

# Calculate and display the average results
avg_accuracy = np.mean([result['accuracy'] for result in all_fold_results])
avg_f1 = np.mean([result['f1'] for result in all_fold_results])
print(f"\nAverage Accuracy: {avg_accuracy}")
print(f"Average F1-Score: {avg_f1}")

/Users/aaryanshah/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aaryanshah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/aaryanshah/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/aaryanshah/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be rem


Fold 1


Some weights of CustomRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/889 [00:00<?, ?it/s]

{'loss': 1.6019, 'grad_norm': 3.533015251159668, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}
{'loss': 1.3696, 'grad_norm': 5.096282482147217, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.79}


...

{'eval_loss': 0.7823296785354614, 'eval_accuracy': 0.7272727272727273, 'eval_f1': 0.7192996340441216, 'eval_runtime': 341.3288, 'eval_samples_per_second': 1.482, 'eval_steps_per_second': 0.094, 'epoch': 2.0}
{'loss': 0.6856, 'grad_norm': 13.266212463378906, 'learning_rate': 1.2e-05, 'epoch': 2.36}
{'loss': 0.7135, 'grad_norm': 10.422714233398438, 'learning_rate': 1.4e-05, 'epoch': 2.76}


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.7482283115386963, 'eval_accuracy': 0.7292490118577075, 'eval_f1': 0.7238912493861531, 'eval_runtime': 23.2897, 'eval_samples_per_second': 21.726, 'eval_steps_per_second': 1.374, 'epoch': 3.0}
{'loss': 0.6302, 'grad_norm': 11.530143737792969, 'learning_rate': 1.6000000000000003e-05, 'epoch': 3.15}
{'loss': 0.5706, 'grad_norm': 22.487537384033203, 'learning_rate': 1.8e-05, 'epoch': 3.54}
{'loss': 0.5901, 'grad_norm': 29.152694702148438, 'learning_rate': 2e-05, 'epoch': 3.94}


  0%|          | 0/32 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# perform hyperparameter tuning

# Define hyperparameter search space
param_space = {
    'learning_rate': [2e-5, 3e-5, 4e-5],
    'num_train_epochs': [3, 4, 5],
    'per_device_train_batch_size': [8, 16, 32]
}

# Perform hyperparameter tuning
best_accuracy = 0
best_params = None

for params in ParameterGrid(param_space):
    print(f"Running with parameters: {params}")

    # Create datasets
    train_dataset = CustomTextDataset(
        tokenized_texts,
        df_cleaned['labels'].tolist(),
        df_cleaned['core_count'].tolist(),
        df_cleaned['emerging_count'].tolist()
    )
    test_dataset = CustomTextDataset(
        tokenized_texts,
        df_cleaned['labels'].tolist(),
        df_cleaned['core_count'].tolist(),
        df_cleaned['emerging_count'].tolist()
    )

    # Initialize a fresh model for each fold with weighted loss
    model = CustomRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

    # Trainer with early stopping callback
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            output_dir='./results',  # Directory for storing logs and model checkpoints
            num_train_epochs=params['num_train_epochs'],  # Train for 7 epochs
            per_device_train_batch_size=params['per_device_train_batch_size'],  # Smaller batch size to manage GPU memory
            per_device_eval_batch_size=16,  # Batch size for evaluation
            warmup_steps=500,  # Number of warmup steps for learning rate scheduler
            warmup_ratio=0.1,  # Alternatively, set warmup as a ratio of total steps
            weight_decay=0.01,  # Weight decay for regularization
            logging_dir='./logs',  # Directory for storing logs
            logging_steps=50,  # Log model predictions every 50 steps
            evaluation_strategy="epoch",  # Evaluate model at the end of each epoch
            learning_rate=params['learning_rate'],  # Higher initial learning rate
            lr_scheduler_type='linear',  # Learning rate scheduler
            fp16=True,  # Enable mixed precision training
            gradient_accumulation_steps=2,  # Use gradient accumulation for larger effective batch size
            save_strategy="epoch",  # Save the model at the end of each epoch
            load_best_model_at_end=True,  # Load the best model at the end of training
            metric_for_best_model="eval_loss",  # Use evaluation loss to determine the best model
            save_total_limit=2  # Limit the total amount of checkpoints; older ones are deleted
        ),
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=lambda p: {
            'accuracy': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1)),
            'f1': classification_report(p.label_ids, np.argmax(p.predictions, axis=1), target_names=[str(i+1) for i in range(5)], output_dict=True)['weighted avg']['f1-score']
        },
        callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
    )
    
    # Train the model
    trainer.train()

    # Evaluate the model
    eval_result = trainer.evaluate()

    # Save the best model
    if eval_result['eval_accuracy'] > best_accuracy:
        best_accuracy = eval_result['eval_accuracy']
        best_params = params
        
    print(f"Fold {fold} - Accuracy: {eval_result['eval_accuracy']}")
    print(f"Fold {fold} - F1-Score: {eval_result['eval_f1']}")

    all_fold_results.append({
        'fold': fold,
        'accuracy': eval_result['eval_accuracy'],
        'f1': eval_result['eval_f1']
    })

    fold += 1
    
print(f"Best parameters: {best_params}")
print(f"Best accuracy: {best_accuracy}")

# Train the final model with the best hyperparameters
model = CustomRobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=5)

# Trainer with early stopping callback
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='./results',  # Directory for storing logs and model checkpoints
        num_train_epochs=best_params['num_train_epochs'],  # Train for 7 epochs
        per_device_train_batch_size=best_params['per_device_train_batch_size'],  # Smaller batch size to manage GPU memory
        per_device_eval_batch_size=16,  # Batch size for evaluation
        warmup_steps=500,  # Number of warmup steps for learning rate scheduler
        warmup_ratio=0.1,  # Alternatively, set warmup as a ratio of total steps
        weight_decay=0.01,  # Weight decay for regularization
        logging_dir='./logs',  # Directory for storing logs
        logging_steps=50,  # Log model predictions every 50 steps
        evaluation_strategy="epoch",  # Evaluate model at the end of each epoch
        learning_rate=best_params['learning_rate'],  # Higher initial learning rate
        lr_scheduler_type='linear',  # Learning rate scheduler
        fp16=True,  # Enable mixed precision training
        gradient_accumulation_steps=2,  # Use gradient accumulation for larger effective batch size
        save_strategy="epoch",  # Save the model at the end of each epoch
        load_best_model_at_end=True,  # Load the best model at the end of training
        metric_for_best_model="eval_loss",  # Use evaluation loss to determine the best model
        save_total_limit=2  # Limit the total amount of checkpoints; older ones are deleted
    ),
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {
        'accuracy': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1)),
        'f1': classification_report(p.label_ids, np.argmax(p.predictions, axis=1), target_names=[str(i+1) for i in range(5)], output_dict=True)['weighted avg']['f1-score']
    },
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)